<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li>

# Отток клиентов

Маркетологи некоторого банка обратили внимание на отток клиентов и посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

На предоставленных исторических данных о поведении клиентов и расторжении договоров с банком необходимо спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. 

Ключевая метрика - *F1*-мера, дополнительная - *AUC-ROC*

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
# подгружаю необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.utils import shuffle

from sklearn.metrics import f1_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score 

data = pd.read_csv('/datasets/Churn.csv')
display(data)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [3]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
# вижу пропуски в столбце Tenure
print(data['Tenure'].value_counts().sort_values(ascending=False))
print('Среднее значение', data['Tenure'].mean())
print('Медиана', data['Tenure'].median())

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64
Среднее значение 4.997690023099769
Медиана 5.0


In [7]:
# так как особой взаимосвязи не видно, заменю пропуски случайные значения, находящиеся в границах столбца
def func(row):
    if pd.isna(row['Tenure']):
        return random.randint(0,10)
    return row['Tenure']
 
data['Tenure'] = data.apply(func, axis=1)
display(data)
print('Среднее значение', data['Tenure'].mean())
print('Медиана', data['Tenure'].median())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


Среднее значение 5.0009
Медиана 5.0


In [8]:
# так как значения среднего и медианы сильно не изменились, замена пропусков совершена правильно
# проверяю какие столбцы можно закодировать 
print(sorted(data['Geography'].unique()))
print(sorted(data['Gender'].unique()))

['France', 'Germany', 'Spain']
['Female', 'Male']


In [9]:
# проверяем данные на наличие дубликатов
print(data.duplicated().sum())

0


In [10]:
# так как фамилии, порядкой номер строки и CustomerId не влияют на дальнейшую работу, удаляю эти столбцы
data1 = data.drop(['Surname', 'CustomerId', 'RowNumber'], axis=1)
print(data1.shape)

(10000, 11)


In [11]:
# подготавливая признаки, выбираю способ кодирования One-hot_encoding
data_ohe = pd.get_dummies(data1, drop_first=True)
data_ohe.shape
display(data_ohe)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,1,0,1


In [12]:
# перехожу к подготовке данных
# разбиваю выборку на обучающую, валидационную
data_train_test, data_valid = train_test_split(data_ohe, test_size=0.2, random_state=12345)
data_train, data_test = train_test_split(data_train_test, test_size=0.25, random_state=12345)
display(data_train.shape)
display(data_valid.shape)
display(data_test.shape)

(6000, 12)

(2000, 12)

(2000, 12)

In [13]:
target = data['Exited'] # целевой признак
features = data.drop(['Exited'], axis=1) # признаки

target_train = data_train['Exited'] 
features_train = data_train.drop(['Exited'], axis=1) 

target_valid = data_valid['Exited']
features_valid = data_valid.drop(['Exited'], axis=1)

target_test = data_test['Exited']
features_test = data_test.drop(['Exited'], axis=1)

print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
print(target_train.shape)
print(target_valid.shape)
print(target_test.shape)


(6000, 11)
(2000, 11)
(2000, 11)
(6000,)
(2000,)
(2000,)


In [14]:
# провожу стандартизацию признаков
numeric = ['CreditScore',
           'Age',
           'Tenure',
           'Balance',
           'NumOfProducts',
           'EstimatedSalary'] # количественные признаки, кодируемые в категориальные переменные

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(6000, 11)
(2000, 11)
(2000, 11)


**Вывод**

1. Получила данные, состоящие из 10000 строк и 14 стобцов.

    *Признаки* 
* RowNumber — индекс строки в данных
* CustomerId — уникальный идентификатор клиента
* Surname — фамилия
* CreditScore — кредитный рейтинг
* Geography — страна проживания
* Gender — пол
* Age — возраст
* Tenure — сколько лет человек является клиентом банка
* Balance — баланс на счёте
* NumOfProducts — количество продуктов банка, используемых клиентом
* HasCrCard — наличие кредитной карты
* IsActiveMember — активность клиента
* EstimatedSalary — предполагаемая зарплата

    *Целевой признак*
 
* Exited — факт ухода клиента

2. Провела предобработку данных

- Пропуски обнаружены были только в столбце Tenure, заменили их на медиану, расчитанную по региону
- Дубликатов не обнаружено
- Удален столбец с фамилиями, так как при кодировании и избегании дамми-ловушек этот столбец превращает данные в датафрейм с более 2000 столбцов. Тем более как признак фамилии не влияют на отток клиентов.

3. Закодировала получившийся датасет с помощью One-hot-encoder (так как переводим количественные признаки в категориальные)

4. Разбила выборку на обучающую и валидационную с помощью train_test_split, обучающая выборка составила 75%

5. Стандартизировала признаки с помощью StandardScaler


## Исследование задачи

In [15]:
# DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print('F1 =', f1_score(target_valid, predictions_valid))

# сравниваю f1 c auc-roc
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc =', auc_roc)

F1 = 0.48584905660377364
auc_roc = 0.6728770484359158


In [16]:
# DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print('F1 =', f1_score(target_valid, predictions_valid))

# для модели DecisionTreeRegressor нельзя посчитать auc_roc, так как задача заключается в классификации

F1 = 0.5064177362893816


In [17]:
# RandomForestClassifier
# сначала определим наилучшую глубину
best_result = 0
best_depth = 0
for depth in range(1, 16, 1):
    model = RandomForestClassifier(n_estimators=40,
                                   max_depth=depth,
                                   random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    print('max_depth =', depth, 'f1_score =', result)
    if result > best_result:
        best_result = result
        best_depth = depth
print('best_depth =', best_depth, 'best f1_score =', result)

max_depth = 1 f1_score = 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


max_depth = 2 f1_score = 0.1515151515151515
max_depth = 3 f1_score = 0.21030927835051547
max_depth = 4 f1_score = 0.3181818181818182
max_depth = 5 f1_score = 0.4230769230769231
max_depth = 6 f1_score = 0.506578947368421
max_depth = 7 f1_score = 0.5373608903020667
max_depth = 8 f1_score = 0.5496894409937888
max_depth = 9 f1_score = 0.5592705167173253
max_depth = 10 f1_score = 0.5662650602409639
max_depth = 11 f1_score = 0.545727136431784
max_depth = 12 f1_score = 0.5650224215246636
max_depth = 13 f1_score = 0.5490196078431373
max_depth = 14 f1_score = 0.5621301775147929
max_depth = 15 f1_score = 0.5557206537890045
best_depth = 10 best f1_score = 0.5557206537890045


In [18]:
# RandomForestClassifier
# теперь при наилучшей глубине выберем наилучшее количество деревьев
best_result = 0
best_est = 0
for est in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators=est,
                                   max_depth=best_depth,
                                   random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    print('est =', est, 'f1_score =', result)
    if result > best_result:
        best_result = result
        best_est = est
print('best_est =', best_est, 'best f1_score =', result)

est = 10 f1_score = 0.5866666666666666
est = 20 f1_score = 0.5705705705705706
est = 30 f1_score = 0.571001494768311
est = 40 f1_score = 0.5662650602409639
est = 50 f1_score = 0.5627836611195159
est = 60 f1_score = 0.5658093797276853
est = 70 f1_score = 0.5597579425113465
est = 80 f1_score = 0.5597579425113465
est = 90 f1_score = 0.5671191553544496
est = 100 f1_score = 0.5679758308157099
best_est = 10 best f1_score = 0.5679758308157099


In [19]:
# RandomForestClassifier
# после определения гиперпараметров с лучшими результатами проверяю auc_roc
model = RandomForestClassifier(n_estimators=60,
                                   max_depth=12,
                                   random_state=12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = f1_score(target_valid, predictions_valid)
print('f1_score =', result)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc =', auc_roc)

f1_score = 0.5645645645645645
auc_roc = 0.8635180021171079


Принимает наилучший результат модель RandomForestClassifier, f1_score = 0.58 при depth = 12 и est = 60, auc_roc =0.86

In [20]:
# LogisticRegressior
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print('F1 =', f1_score(target_valid, predictions_valid))

# сравниваю f1 c auc-roc
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc =', auc_roc)

F1 = 0.3006993006993007
auc_roc = 0.7581092528931574


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Вывод**

Были проверены 4 модели без учета дисбаланса, у которых есть возможность определить f1-score:

* DecisionTreeClassifier: f1_score = 0.58
* DecisionTreeRegressor: f1_score = 0.51
* RandomForestClassifier: f1_score = 0.56
* LogisticRegression: f1_score = 0.3

У модели DecisionTreeRegressor нельзя определить auc-roc.
* DecisionTreeClassifier: auc_roc = 0.69
* RandomForestClassifier: auc_roc = 0.86
* LogisticRegression: auc_roc = 0.76

По результатам метрик F1 и AUC-ROC самые лучшие результаты показывает модель RandomForestClassifier.

Проверим модели с учетом дисбаланса.

## Борьба с дисбалансом

In [21]:
# DecisionTreeClassifier
model = DecisionTreeClassifier(class_weight='balanced')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print('F1 =', f1_score(target_valid, predictions_valid))


# сравниваю f1 c auc-roc
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc =', auc_roc)

F1 = 0.5024271844660194
auc_roc = 0.6819946074789591


In [22]:
# RandomForestClassifier
# сначала определим наилучшую глубину
best_result = 0
best_depth = 0
for depth in range(1, 16, 1):
    model = RandomForestClassifier(n_estimators=40,
                                   max_depth=depth,
                                   random_state=12345,
                                   class_weight='balanced')
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    print('max_depth =', depth, 'f1_score =', result)
    if result > best_result:
        best_result = result
        best_depth = depth
print('best_depth =', best_depth, 'best f1_score =', result)

# сравниваю f1 c auc-roc
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc =', auc_roc)

max_depth = 1 f1_score = 0.5353283458021613
max_depth = 2 f1_score = 0.5834109972041007
max_depth = 3 f1_score = 0.5858951175406871
max_depth = 4 f1_score = 0.608695652173913
max_depth = 5 f1_score = 0.6147783251231528
max_depth = 6 f1_score = 0.6170634920634921
max_depth = 7 f1_score = 0.6343975283213181
max_depth = 8 f1_score = 0.6472491909385113
max_depth = 9 f1_score = 0.64293659621802
max_depth = 10 f1_score = 0.6408450704225352
max_depth = 11 f1_score = 0.6202531645569621
max_depth = 12 f1_score = 0.6191709844559584
max_depth = 13 f1_score = 0.6034712950600801
max_depth = 14 f1_score = 0.5673758865248227
max_depth = 15 f1_score = 0.5689655172413793
best_depth = 8 best f1_score = 0.5689655172413793
auc_roc = 0.8562145752905812


In [23]:
# RandomForestClassifier
# теперь при наилучшей глубине выберем наилучшее количество деревьев
best_result = 0
best_est = 0
for est in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators=est,
                                   max_depth=best_depth,
                                   random_state=12345,
                                   class_weight='balanced')
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    print('est =', est, 'f1_score =', result)
    if result > best_result:
        best_result = result
        best_est = est
print('best_est =', best_est, 'best f1_score =', result)

# сравниваю f1 c auc-roc
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc =', auc_roc)

est = 10 f1_score = 0.6229860365198711
est = 20 f1_score = 0.6464864864864864
est = 30 f1_score = 0.6416309012875536
est = 40 f1_score = 0.6472491909385113
est = 50 f1_score = 0.6537216828478964
est = 60 f1_score = 0.644468313641246
est = 70 f1_score = 0.6509635974304069
est = 80 f1_score = 0.6466809421841542
est = 90 f1_score = 0.6459893048128342
est = 100 f1_score = 0.6501614639397202
best_est = 50 best f1_score = 0.6501614639397202
auc_roc = 0.8690385620340912


Модель RandomForestClassifier с учетом дисбаланса принимает наилучший результат f1_score = 0.65 при depth = 8 и est = 60, auc_roc=0.87

In [24]:
# LogisticRegressior
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print('F1 =', f1_score(target_valid, predictions_valid))

# сравниваю f1 c auc-roc
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc =', auc_roc)

F1 = 0.5029337803855826
auc_roc = 0.7638188934761216


**Вывод**

С учетом дисбаланса были рассмотрены 3 модели, у которых также была посчитала метрика auc-roc:

* DecisionTreeClassifier: 
        -F1 = 0.5053635280095352
        -auc_roc = 0.6846707688734515
        
* RandomForestClassifier:
        -F1_score = 0.6449511400651465
        -auc_roc = 0.8670063170808326
        (при гиперпараметрах depth = 8 и est = 60)
        
* LogisticRegression:
        -F1 = 0.5050505050505051
        -auc_roc = 0.7640213735593766

Наилучший результат с учетом дисбаланса также показала модель RandomForestClassifier:

f1_score = 0.64 при depth = 8 и est = 60
auc_roc = 0.87 - так же наилучший результат

Теперь пробую увеличить выборку

In [25]:
# RandomForestClassifier
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, 
                                                   target_upsampled, 
                                                   random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

model = RandomForestClassifier(random_state=12345, max_depth=8, n_estimators=90)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5195339273474983


In [26]:
# LogisticRegression
features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.4261943102522813


**Вывод**

С увеличением выборки результат ухучшился.

Пробую уменьшить выборку

In [27]:
# RandomForestClassifier
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

model = RandomForestClassifier(random_state=12345, max_depth=8, n_estimators=90)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.4771634615384615


In [28]:
# LogisticRegression
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.42336548767416937


**Вывод**

Аналогично с уменьшением выборки также результат ухучшился. Останавливаюсь на сбалансированном случайном лесе.
И построю для него ROC-кривую

In [29]:
my_model = RandomForestClassifier(n_estimators=60,
                               max_depth=8,
                               random_state=12345,
                               class_weight='balanced')
my_model.fit(features_train, target_train)
predictions_valid = my_model.predict(features_valid)
result = f1_score(target_valid, predictions_valid)
print('f1_score =', result)

probabilities_valid = my_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc =', auc_roc)

fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)

plt.figure()

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')

# ROC-кривая случайной модели
plt.plot([0, 1], [0, 1], linestyle='--')

plt.show()

f1_score = 0.644468313641246
auc_roc = 0.8685517165397941


Валидационная выборка показала результат f1 = 0.65, auc_roc = 0.87 - самые высокие результаты по всем проверенным моделям. 
Теперь тестирую наилучшую модель RandomForestClassifier на тестовой выборке, так же проверяя наилучшие гиперпараметры

In [30]:
# Пробую для удобства определить features importance
importances = my_model.feature_importances_
my_model_importances = pd.DataFrame(importances, index=features_train.columns)
my_model_importances.rename(columns = {0 : 'values'}, inplace = True)
my_model_importances = my_model_importances.reset_index()
display(my_model_importances)

,index,values
0,CreditScore,0.059609
1,Age,0.355651
2,Tenure,0.033565
3,Balance,0.105433
4,NumOfProducts,0.214192
5,HasCrCard,0.006768
6,IsActiveMember,0.070223
7,EstimatedSalary,0.061971
8,Geography_Germany,0.061625
9,Geography_Spain,0.007264


## Тестирование модели

In [31]:
# оставляю модель RandomForestClassifier с гиперпараметрами 
# est = 60, depth = 8, class_weight = 'balanced'
# и проверяю ее на тестовой выборки

model = RandomForestClassifier(n_estimators=60,
                               max_depth=8,
                               random_state=12345,
                               class_weight='balanced')
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = f1_score(target_test, predictions_test)
print('f1_score =', result)

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('auc_roc =', auc_roc)

f1_score = 0.5920745920745921
auc_roc = 0.517310715460827


In [32]:
fpr, tpr, thresholds = roc_curve(target_test, probabilities_one_test)

plt.figure()

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')

# ROC-кривая случайной модели
plt.plot([0, 1], [0, 1], linestyle='--')

plt.show()

## Общий вывод

**Цель проекта**

Нужно спрогнозировать, уйдёт клиент из "Бета-Банка" в ближайшее время или нет, ориентируясь на исторические данные о поведении клиентов и расторжении договоров с банком.
Нужно постройть модель с предельно большим значением F1-меры и проверить F1-меру на тестовой выборке. Дополнительно измеряить AUC-ROC, сравнивая её значение с F1-мерой.

**Полученные данные**

    В нашем распоряжении данные о поведении клиентов, состоящие из 10000 строк и 14 стобцов.

   *Признаки*
    
   * RowNumber — индекс строки в данных
   * CustomerId — уникальный идентификатор клиента
   * Surname — фамилия
   * CreditScore — кредитный рейтинг
   * Geography — страна проживания
   * Gender — пол
   * Age — возраст
   * Tenure — сколько лет человек является клиентом банка
   * Balance — баланс на счёте
   * NumOfProducts — количество продуктов банка, используемых клиентом
   * HasCrCard — наличие кредитной карты
   * IsActiveMember — активность клиента
   * EstimatedSalary — предполагаемая зарплата

   *Целевой признак*
 
   * Exited — факт ухода клиента

    Провела предобработку данных

       - Пропуски обнаружены были только в столбце Tenure, заменили их на медиану, расчитанную по региону
       - Дубликатов не обнаружено
       - Удален столбец с фамилиями, так как при кодировании и избегании дамми-ловушек этот столбец превращает данные в датафрейм с более 2000 столбцов. Тем более как признак фамилии не влияют на отток клиентов.

    Закодировала получившийся датасет с помощью One-hot-encoder (так как переводим количественные признаки в категориальные)

    Разбила выборку на обучающую и валидационную с помощью train_test_split, обучающая выборка составила 75%

    Стандартизировала признаки с помощью StandardScaler


**Выборки**

Разделили данные на три выборки: обучающую, валидационную и тестовую. От общих данных обучающая составила 60%, валидационная и тестовая по 20%.

**Исследование разных моделей без учета дисбаланса**

Были проверены 4 модели без учета дисбаланса, у которых есть возможность определить f1-score:

* DecisionTreeClassifier: f1_score = 0.58
* DecisionTreeRegressor: f1_score = 0.51
* RandomForestClassifier: f1_score = 0.56
* LogisticRegression: f1_score = 0.3

У модели DecisionTreeRegressor нельзя определить auc-roc.
* DecisionTreeClassifier: auc_roc = 0.69
* RandomForestClassifier: auc_roc = 0.86
* LogisticRegression: auc_roc = 0.76

**Исследование разных моделей с учетом дисбаланса**
С учетом дисбаланса были рассмотрены 3 модели, у которых также была посчитала метрика auc-roc:

* DecisionTreeClassifier: 
        -F1 = 0.5053635280095352
        -auc_roc = 0.6846707688734515
        
* RandomForestClassifier:
        -F1_score = 0.6449511400651465
        -auc_roc = 0.8670063170808326
        (при гиперпараметрах depth = 8 и est = 60)
        
* LogisticRegression:
        -F1 = 0.5050505050505051
        -auc_roc = 0.7640213735593766

Наилучший результат с учетом дисбаланса также показала модель RandomForestClassifier:

f1_score = 0.64 при depth = 8 и est = 60
auc_roc = 0.87 - так же наилучший результат

С увеличением и с уменьшением выборки результаты ухучшились, поэтому оставила модель RandomForestClassifier:
        - F1 = 0.64 (best_est = 60, best_depth = 8)
        - auc_roc = 0.87

**Проверка модели на тестовой выборке**

Через модель, выбранную в предыдущем пункте, прогнала тестовую выборку, которая показала следующие результаты.

 - f1_score = 0.5924217844727694
 - auc_roc = 0.5151950584865502

**Общий вывод**

С помощью исследования построили модель случайного леса, которая лучше других справилась с задачей классификации, следовательно, ее стоит внедрить для успешного анализа поведения клиентов и определения, расторгнет договор клиент в ближайшее время или нет.